<a href="https://colab.research.google.com/github/tjwei/NCTU_Private_DeepLearning/blob/master/Secret_Sharing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Additive Secret Sharing

Most of the material taken from https://github.com/udacity/private-ai/blob/master/Section%203%20-%20Securing%20Federated%20Learning.ipynb and https://github.com/udacity/private-ai/blob/master/Section%204%20-%20Encrypted%20Deep%20Learning.ipynb

For more great information about SMPC protocols like this one, visit https://mortendahl.github.io. 

While being able to have a trusted third party is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive.

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [0]:
x = 5
bob_x_share=x
alice_x_share=x

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [0]:
bob_x_share *= 2
alice_x_share *= 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

20

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted).

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [0]:
bob_x_share = 2 * 2
alice_x_share = 3 * 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

This even works for addition between two shared values!!



In [0]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

As you can see, we just added two numbers together while they were still encrypted!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant. I.e.

In [0]:
import random
x = 5

Q = 23740629843760239486723

#bob_x_share = 23552870267 # <- a random number
bob_x_share = random.randint(0,Q-1)# <- random number
alice_x_share = Q - bob_x_share + x
alice_x_share

10280729013724167067673

In [0]:
bob_x_share

3388921462544650408717

In [0]:
(bob_x_share + alice_x_share) % Q

5

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add
In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like ```x_share = (2,5,7)``` if it has 3 shares.



In [0]:
import random
import numpy as np
# try this project here!
default_Q = 293973345475167247070445277780365744413


def encrypt(x, n_shares=3, Q=default_Q):
  x_shares = [random.randint(0,Q-1) for i in range ( n_shares-1)]
  x_shares.append((x - sum(x_shares))%Q)#x-sum(x_shares)會是負的 %Q後會轉正  EX:ㄌ
  return x_shares

def decrypt(x_shares ,Q=default_Q):
  x = sum(x_shares)%Q
  return x
def add(x_shares, y_shares, Q=default_Q):
  # complete the code here
  assert len(x_shares)==len(y_shares) #兩個長度要一樣不然會跳error
  z_shares = [(xi+yi)%Q for xi,yi in zip (x_shares, y_shares)]
  return z_shares





In [0]:
bob_x_share = random.randint(0,Q-1)# <- random number
bob_x_share  

3997167575567088445604

In [0]:
encrypt(5)

[289127312595553595506721354894230675013,
 95916489093130896842426468782843992971,
 202902889261650001791742731883656820847]

In [0]:
decrypt(encrypt(5))

5

In [0]:
add(encrypt(5), encrypt(6))

[284494441429172466401328618261121364142,
 99525845925069179089611639683520536942,
 203926403596092848649950297616089587753]

In [0]:
# a few simple tests
assert decrypt(encrypt(113))==113
assert decrypt(add(encrypt(5), encrypt(6)))==11

# Project: Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [0]:
# complete the following code
import random
import numpy as np

BASE = 10

PRECISION_INTEGRAL = 8
PRECISION_FRACTIONAL = 8

PRECISION = PRECISION_INTEGRAL + PRECISION_FRACTIONAL
default_Q = 293973345475167247070445277780365744413
assert(default_Q > BASE**PRECISION)

# note that a rational might be negative
# encode and decode rationals to field_element
def encode(rational, Q=default_Q):
    # complete code here
    field_element = int(rational*10**PRECISION_FRACTIONAL)%Q
    return field_element

def decode(field_element, Q=default_Q):
    # complete code here
    field_element%=Q
    if field_element > Q//2:
      field_element-=Q
    rational = field_element/10**PRECISION_FRACTIONAL
    return rational


8

In [0]:
field_element = encode(5.5)
field_element

293973345475167247070445277779815744413

In [0]:
# testing
x = encrypt(encode(5.5))
y = encrypt(encode(2.3))
z = add(x,y)
decode(decrypt(z))

7.79999999

# Lesson: Encrypted Subtraction and Public/Scalar Multiplication

In [0]:
field = 23740629843760239486723

In [0]:
x = 5

bob_x_share = 2372385723 # random number
alices_x_share = field - bob_x_share + x

In [0]:
(bob_x_share + alices_x_share) % field

5

In [0]:
field = 10

x = 5

bob_x_share = 8
alice_x_share = field - bob_x_share + x

y = 1

bob_y_share = 9
alice_y_share = field - bob_y_share + y

In [0]:
((bob_x_share + alice_x_share) - (bob_y_share + alice_y_share)) % field

In [0]:
((bob_x_share - bob_y_share) + (alice_x_share - alice_y_share)) % field

4

In [0]:
bob_x_share + alice_x_share + bob_y_share + alice_y_share

26

In [0]:
bob_z_share = (bob_x_share - bob_y_share)
alice_z_share = (alice_x_share - alice_y_share)

In [0]:
(bob_z_share + alice_z_share) % field

4

In [0]:
def sub(a, b):
    c = list()
    for i in range(len(a)):
        c.append((a[i] - b[i]) % Q)
    return tuple(c)

In [0]:
field = 10

x = 5

bob_x_share = 8
alice_x_share = field - bob_x_share + x

y = 1

bob_y_share = 9
alice_y_share = field - bob_y_share + y

In [0]:
bob_x_share + alice_x_share

15

In [0]:
bob_y_share + alice_y_share

11

In [0]:
((bob_y_share * 3) + (alice_y_share * 3)) % field

3

In [0]:
def imul(a, scalar):
    
    # logic here which can multiply by a public scalar
    
    c = list()
    
    for i in range(len(a)):
        c.append((a[i] * scalar) % Q)
        
    return tuple(c)

In [0]:
x = encrypt(encode(5.5))
x

NameError: ignored

In [0]:
z = imul(x, 3)

In [0]:
decode(decrypt(z))

# Lesson: Encrypted Computation in PySyft

In [3]:
!pip install syft

     |████████████████████████████████| 317kB 5.2MB/s 
     |████████████████████████████████| 256kB 18.1MB/s 
     |████████████████████████████████| 81kB 23.5MB/s 
     |████████████████████████████████| 204kB 36.0MB/s 
     |████████████████████████████████| 2.1MB 42.7MB/s 
     |████████████████████████████████| 399kB 46.2MB/s 
     |████████████████████████████████| 460kB 45.1MB/s 
     |████████████████████████████████| 266kB 44.2MB/s 
     |████████████████████████████████| 51kB 19.3MB/s 
     |████████████████████████████████| 122kB 44.5MB/s 
  Created wheel for zstd: filename=zstd-1.4.3.2-cp36-cp36m-linux_x86_64.whl size=1076252 sha256=692b479ea95aa8acc7a9c6ec64421e03ffbe9babfbea2e0dcdff678543089893
  Stored in directory: /root/.cache/pip/wheels/f2/df/a8/405bafcabba88a18c8763694e79177e2a1bbc65ac0f6b3d728
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp36-cp36m-linux_x86_64.whl size=44104 sha256=cf9559488da5d83c0f8adb98b53e10ce7e427ba0046392f37787234a227e19c1
  Stored in di

In [2]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim

ModuleNotFoundError: ignored

In [0]:
bob = sy.VirtualWorker(hook, id="bob").add_worker(sy.local_worker)
alice = sy.VirtualWorker(hook, id="alice").add_worker(sy.local_worker)
secure_worker = sy.VirtualWorker(hook, id="secure_worker").add_worker(sy.local_worker)

In [0]:
x = th.tensor([1,2,3,4])
y = th.tensor([2,-1,1,0])

In [0]:
x = x.share(bob, alice, crypto_provider=secure_worker)

In [0]:
y = y.share(bob, alice, crypto_provider=secure_worker)

In [0]:
z = x + y
z.get()

In [0]:
z = x - y
z.get()

In [0]:
z = x * y
z.get()

In [0]:
z = x > y
z.get()

In [0]:
z = x < y
z.get()

In [0]:
z = x == y
z.get()

In [0]:
x = th.tensor([1,2,3,4])
y = th.tensor([2,-1,1,0])

x = x.fix_precision().share(bob, alice, crypto_provider=secure_worker)
y = y.fix_precision().share(bob, alice, crypto_provider=secure_worker)

In [0]:
z = x + y
z.get().float_precision()

In [0]:
z = x - y
z.get().float_precision()

In [0]:
z = x * y
z.get().float_precision()

In [0]:
z = x > y
z.get().float_precision()

In [0]:
z = x < y
z.get().float_precision()

In [0]:
z = x == y
z.get().float_precision()

# Multiplication

suppose $a \cdot b=c$ and we want to compute $x \cdot y $ 

Let $\alpha = x-a$, 
$\beta = y-b$

$xy = \alpha \beta + ay + bx - c $


# Lesson: Encrypted Deep Learning in PyTorch

### Train a Model

In [1]:
from torch import nn
from torch import optim
import torch.nn.functional as F

# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 20)
        self.fc2 = nn.Linear(20, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

# A Toy Model
model = Net()

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data)
        
train()

NameError: ignored

In [0]:
model(data)

tensor([[ 0.1045],
        [-0.0507],
        [ 0.9947],
        [ 0.9916]], grad_fn=<AddmmBackward>)

## Encrypt the Model and Data

In [0]:
encrypted_model = model.fix_precision().share(alice, bob, crypto_provider=secure_worker)

In [0]:
list(encrypted_model.parameters())

In [0]:
encrypted_data = data.fix_precision().share(alice, bob, crypto_provider=secure_worker)

In [0]:
encrypted_data

In [0]:
encrypted_prediction = encrypted_model(encrypted_data)

In [0]:
encrypted_prediction.get().float_precision()

# Project
Build an secrect sharing scheme for english text. 

In [0]:
import random
import numpy as np
import math
# try this project here!
default_Q = 293973345475167247070445277780365744413


def encrypt(x, n_shares=3, Q=default_Q):
  x_shares = [random.randint(0,Q-1) for i in range ( n_shares-1)]
  x_shares.append((x - sum(x_shares))%Q)#x-sum(x_shares)會是負的 %Q後會轉正  EX:ㄌ
  return x_shares

def decrypt(x_shares ,Q=default_Q):
  x = sum(x_shares)%Q
  return x

def add(x_shares, y_shares, Q=default_Q):
  # complete the code here
  assert len(x_shares)==len(y_shares) #兩個長度要一樣不然會跳error
  z_shares = [(xi+yi)%Q for xi,yi in zip (x_shares, y_shares)]
  return z_shares

def encrypt_text(string_id):
  encrypttext=[]
  for i in range(len(string_id)):
    encrypttext.append(encrypt(string_id[i]))
  return encrypttext


def decrypt_text(encrypttext):
  string_id = []
  for j in range(len(encrypttext)):
    string_id.append(decrypt(encrypttext[j]))
  return string_id


In [0]:
str ="""
You may write me down in history

With your bitter, twisted lies,

You may tread me in the very dirt

But still, like dust, I’ll rise.
""" 
def turn_to_string_id(str):
  string_number = 0
  string_id = []
  str_list = str.split()
  #print (len(str_list))
  for i in range(len(str_list)):
    print(str_list[i])
    for j in range(len(str_list[i])):
      string_number +=  ord(str_list[i][j])*10**(4*(j))
      #print("第{}個字串,第{}個字是{},轉成數字後是{}".format(i+1,j+1,str_list[i][j],ord(str_list[i][j]))) 
      #print(ord(str_list[i][j])*10**(3*(j)))
    string_id.append(string_number) 
    string_number = 0
    #print("第{}個字串轉成數字後是:{}".format(i+1,string_id[i]))
  #print(string_id)   
  return string_id

def turn_back_string(string_id):
  text =""
  for i in range(len(string_id)):
    word =""
    c= 0
    e =string_id[i]
    while e>1:
      e/=10
      c+=1
    length = math.ceil(c/4)
    for j in range(length):
      a=string_id[i]//10**(4*(length-1))
      b=chr(a)
      string_id[i] = string_id[i] - a*10**(4*(length-1))
      length-=1
      word+=b
    f=word[::-1]
    text=text+f+' '
  return text

In [0]:
turn_to_string_id(str)


In [0]:
encrypt_text(string_id)

In [0]:
decrypt_text(encrypt_text(string_id))

In [0]:
print(turn_back_string(decrypt_text(encrypt_text(string_id))))

You may write me down in history With your bitter, twisted lies, You may tread me in the very dirt But still, like dust, I’ll rise. 


In [0]:
print(ord('’'))

8217


In [0]:
# test
text ="""
You may write me down in history

With your bitter, twisted lies,

You may tread me in the very dirt

But still, like dust, I’ll rise.
"""

decrypt_text(encrypt_text(text)) == text